# `Parsing JSON & Nested JSON Examples`: Python

# <font color=red>Mr Fugu Data Science</font>

# (◕‿◕✿)

# Purpose & Outcome:

+ Show different techiques to handle different problems (`4 different examples`)

`I want this to be like a nuts and bolts idea of: if you see nasty data show some ways of starting parsing it.`

In [1]:
import pandas as pd
import json
import datetime


# ` ¯\_(ツ)_/¯ ` Here are a few examples I have done previously and want to condense skills for you here.

`--------------------------------`

**Ex 1. )**`'entities': {'hashtags': [],
   'symbols': [],
   'user_mentions': [{'screen_name': 'jonfavs',
     'name': 'Jon Favreau',
     'id': 130496027,
     'id_str': '130496027',
     'indices': [3, 11]}],
   'urls': []}`

`--------------------------------`

**Ex 2. )**`'entities': {'url': {'urls': [{'url': 'https://t.co/3bfMaxi5rZ',
       'expanded_url': 'http://pommeit.com.au',
       'display_url': 'pommeit.com.au',
       'indices': [0, 23]}]}`
       
`---------------------------------`

**Ex 3. )**  `
{'candidate': {'first_name': 'Margaret',
   'last_name': 'Mcdonald',
   'skills': ['skLearn', 'Java', 'R', 'SQL', 'Spark', 'C++'],
   'state': 'AL',
   'specialty': 'Database',
   'experience': 'Mid',
   'relocation': 'no'},
  'HR_related': {'hire_date': [datetime.date(2018, 4, 10)],
   'Salary': [129784],
   'healthcare': ['yes']}}`
   
`---------------------------------`   
   
**Ex 4. )** : `
[('Margaret',
  'Mcdonald',
  ['skLearn', 'Java', 'R', 'SQL', 'Spark', 'C++'],
  'AL',
  ({'specialty': 'Database'}, {'experience': 'Mid'}),
  'no'),
 ('Michael',
  'Carter',
  ['TensorFlow', 'R', 'Spark', 'MongoDB', 'C++', 'SQL'],
  'AR',
  ({'specialty': 'Statistics'}, {'experience': 'Senior'}),
  'yes')]
`

# `json_normalize()`:

+ Normalize, semi-structured `JSON` into a flat table

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html

In [431]:
ex_one={'entities': {'hashtags': [],
   'symbols': [],
   'user_mentions': [{'screen_name': 'jonfavs',
     'name': 'Jon Favreau',
     'id': 130496027,
     'id_str': '130496027',
     'indices': [3, 11]}],
   'urls': []}}


df_1=pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in ex_one['entities'].items() ]))

pd.json_normalize(json.loads(df_1.to_json(orient="records")))

/Users/zatoichi59/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()


,hashtags,symbols,urls,user_mentions.screen_name,user_mentions.name,user_mentions.id,user_mentions.id_str,user_mentions.indices
0,None,None,None,jonfavs,Jon Favreau,130496027,130496027,"[3, 11]"


In [432]:
# import datetime

ex_3=[{'candidate': {'first_name': 'Margaret',
   'last_name': 'Mcdonald',
   'skills': ['skLearn', 'Java', 'R', 'SQL', 'Spark', 'C++'],
   'state': 'AL',
   'specialty': 'Database',
   'experience': 'Mid',
   'relocation': 'no'},
  'HR_related': {'hire_date': [datetime.date(2018, 4, 10)],
   'Salary': [129784],
   'healthcare': ['yes']}},{'candidate': {'first_name': 'Michael',
   'last_name': 'Carter',
   'skills': ['TensorFlow', 'R', 'Spark', 'MongoDB', 'C++', 'SQL'],
   'state': 'AR',
   'specialty': 'Statistics',
   'experience': 'Senior',
   'relocation': 'yes'},
  'HR_related': {'hire_date': [datetime.date(2013, 9, 9)],
   'Salary': [176911],
   'healthcare': ['no']}}]


df_3=pd.DataFrame(ex_3)
df_3

,candidate,HR_related
0,"{'first_name': 'Margaret', 'last_name': 'Mcdon...","{'hire_date': [2018-04-10], 'Salary': [129784]..."
1,"{'first_name': 'Michael', 'last_name': 'Carter...","{'hire_date': [2013-09-09], 'Salary': [176911]..."


In [433]:
pd.json_normalize(df_3['HR_related'])

,hire_date,Salary,healthcare
0,[2018-04-10],[129784],[yes]
1,[2013-09-09],[176911],[no]


In [438]:
df_3sub=pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in df_3['candidate'].items() ]))
df_3sub.reset_index(col_level=0)
df3_cand=df_3sub.T


df3_cand

,first_name,last_name,skills,state,specialty,experience,relocation
0,Margaret,Mcdonald,"[skLearn, Java, R, SQL, Spark, C++]",AL,Database,Mid,no
1,Michael,Carter,"[TensorFlow, R, Spark, MongoDB, C++, SQL]",AR,Statistics,Senior,yes


In [439]:
# put it all together:
pd.concat([df3_cand,pd.json_normalize(df_3['HR_related'])],axis=1)

,first_name,last_name,skills,state,specialty,experience,relocation,hire_date,Salary,healthcare
0,Margaret,Mcdonald,"[skLearn, Java, R, SQL, Spark, C++]",AL,Database,Mid,no,[2018-04-10],[129784],[yes]
1,Michael,Carter,"[TensorFlow, R, Spark, MongoDB, C++, SQL]",AR,Statistics,Senior,yes,[2013-09-09],[176911],[no]


# Alternate way to solve same problem:

In [437]:
hr_=pd.json_normalize(df_3['HR_related'])
candidates_=pd.json_normalize(df_3['candidate'])
pd.concat([candidates_,hr_],axis=1)

,first_name,last_name,skills,state,specialty,experience,relocation,hire_date,Salary,healthcare
0,Margaret,Mcdonald,"[skLearn, Java, R, SQL, Spark, C++]",AL,Database,Mid,no,[2018-04-10],[129784],[yes]
1,Michael,Carter,"[TensorFlow, R, Spark, MongoDB, C++, SQL]",AR,Statistics,Senior,yes,[2013-09-09],[176911],[no]


`------------------------------`


# `pandas.explode()`:

+ flatten out a column of lists

 What Happens if we us `pd.explode()` on the column of lists
 
`------------------------------`

In [440]:
df3_cand.explode('skills')

,first_name,last_name,skills,state,specialty,experience,relocation
0,Margaret,Mcdonald,skLearn,AL,Database,Mid,no
0,Margaret,Mcdonald,Java,AL,Database,Mid,no
0,Margaret,Mcdonald,R,AL,Database,Mid,no
0,Margaret,Mcdonald,SQL,AL,Database,Mid,no
0,Margaret,Mcdonald,Spark,AL,Database,Mid,no
0,Margaret,Mcdonald,C++,AL,Database,Mid,no
1,Michael,Carter,TensorFlow,AR,Statistics,Senior,yes
1,Michael,Carter,R,AR,Statistics,Senior,yes
1,Michael,Carter,Spark,AR,Statistics,Senior,yes
1,Michael,Carter,MongoDB,AR,Statistics,Senior,yes


# We can Separate into Dummy Variables if it is needed 

In [454]:
tr_df=pd.json_normalize(json.loads(df_3sub.to_json(orient="records"))).T

# correct column names from numeric generic labels:
tr_df.columns=['first_name','last_name','skills','state','experience',
               'specialty','relocate']

# expand skills list as dummy variables
dummy_var=pd.get_dummies(tr_df['skills'].apply(pd.Series).stack()).sum(level=0)

# combining columns to get new DF
pd.concat([tr_df,dummy_var],axis=1)


,first_name,last_name,skills,state,experience,specialty,relocate,C++,Java,MongoDB,R,SQL,Spark,TensorFlow,skLearn
0,Margaret,Mcdonald,"[skLearn, Java, R, SQL, Spark, C++]",AL,Database,Mid,no,1,1,0,1,1,1,0,1
1,Michael,Carter,"[TensorFlow, R, Spark, MongoDB, C++, SQL]",AR,Statistics,Senior,yes,1,0,1,1,1,1,1,0


# Recursive:

In [395]:
ex_2={'entities': {'url': {'urls': [{'url': 'https://t.co/3bfMaxi5rZ',
       'expanded_url': 'http://pommeit.com.au',
       'display_url': 'pommeit.com.au',
       'indices': [0, 23]}]}}}


In [466]:
g=[]
def print_dict(v, prefix=''): 
    if isinstance(v, dict):
        for k, v2 in v.items():
            p2 = "{}.{}".format(prefix, k)
            print_dict(v2, p2)           # recursive call
    elif isinstance(v, list):
        for i, v2 in enumerate(v):
            p2 = "{}{}".format(prefix, i)
            print_dict(v2, p2)           # recursive call
    else:
#         g.append('{} : {}'.format(prefix, repr(v)))
        g.append(['{}'.format(prefix),v])
    
    return g

# data after using the function:
print_dict(ex_2)

[['.candidate.first_name', 'Margaret'],
 ['.candidate.last_name', 'Mcdonald'],
 ['.candidate.skills0', 'skLearn'],
 ['.candidate.skills1', 'Java'],
 ['.candidate.skills2', 'R'],
 ['.candidate.skills3', 'SQL'],
 ['.candidate.skills4', 'Spark'],
 ['.candidate.skills5', 'C++'],
 ['.candidate.state', 'AL'],
 ['.candidate.specialty', 'Database'],
 ['.candidate.experience', 'Mid'],
 ['.candidate.relocation', 'no'],
 ['.HR_related.hire_date0', datetime.date(2018, 4, 10)],
 ['.HR_related.Salary0', 129784],
 ['.HR_related.healthcare0', 'yes']]

In [465]:

df_2=pd.DataFrame.from_dict(print_dict(ex_2),orient='columns').T

df_2.columns = df_2.iloc[0]

# Drop first row which are now our old column names:
df_2.drop(df_2.index[0])

,.candidate.first_name,.candidate.last_name,.candidate.skills0,.candidate.skills1,.candidate.skills2,.candidate.skills3,.candidate.skills4,.candidate.skills5,.candidate.state,.candidate.specialty,.candidate.experience,.candidate.relocation,.HR_related.hire_date0,.HR_related.Salary0,.HR_related.healthcare0
1,Margaret,Mcdonald,skLearn,Java,R,SQL,Spark,C++,AL,Database,Mid,no,2018-04-10,129784,yes


In [456]:
# Drop the dot notation for nested names and keep inner names only:

names_=[]
for i in df_2.columns:
    names_.append(i.split('.')[4])
names

['url', 'expanded_url', 'display_url', 'indices0', 'indices1']

In [457]:
# change column names: 
df_2.columns=names

# drop first row which our old column names
df_2.drop(df_2.index[0])

,url,expanded_url,display_url,indices0,indices1
1,https://t.co/3bfMaxi5rZ,http://pommeit.com.au,pommeit.com.au,0,23


`-------------------------`

# Example:

`-------------------------`

In [459]:
 ex_4=[('Margaret',
  'Mcdonald',
  ['skLearn', 'Java', 'R', 'SQL', 'Spark', 'C++'],
  'AL',
  ({'specialty': 'Database'}, {'experience': 'Mid'}),
  'no'),
 ('Michael',
  'Carter',
  ['TensorFlow', 'R', 'Spark', 'MongoDB', 'C++', 'SQL'],
  'AR',
  ({'specialty': 'Statistics'}, {'experience': 'Senior'}),
  'yes')]
    
df_4=pd.DataFrame(ex_4)
# df_4.head()

# convert DF to a list of values
df_4toLst=pd.DataFrame(df_4[4].values.tolist())

vb=pd.json_normalize(json.loads(df_4toLst.to_json(orient="records")))


In [460]:
# combine the two DF's
combined_=pd.concat([df_4,vb],axis=1)

# change column names:
combined_.columns=['first_name','last_name','skills','state','specialty_exp',
                  'relocate','specialty','experience']

# drop the old column storing nested data
combined_.drop(['specialty_exp'],axis=1)

,first_name,last_name,skills,state,relocate,specialty,experience
0,Margaret,Mcdonald,"[skLearn, Java, R, SQL, Spark, C++]",AL,no,Database,Mid
1,Michael,Carter,"[TensorFlow, R, Spark, MongoDB, C++, SQL]",AR,yes,Statistics,Senior


# Iterating: 

In [461]:
ex_2={'candidate': {'first_name': 'Margaret',
   'last_name': 'Mcdonald',
   'skills': ['skLearn', 'Java', 'R', 'SQL', 'Spark', 'C++'],
   'state': 'AL',
   'specialty': 'Database',
   'experience': 'Mid',
   'relocation': 'no'},
  'HR_related': {'hire_date': [datetime.date(2018, 4, 10)],
   'Salary': [129784],
   'healthcare': ['yes']}}

t=[]
for i in ex_2:
    t.append(ex_2[i])
pd.DataFrame(t)

# the t[0]=key, t[1]=value for a list of tuples/lists
pd.concat([pd.DataFrame.from_dict(t[0],orient='index').T,pd.DataFrame(t[1])],axis=1)
# pd.DataFrame.from_dict(t[0],orient='index').T



,first_name,last_name,skills,state,specialty,experience,relocation,hire_date,Salary,healthcare
0,Margaret,Mcdonald,"[skLearn, Java, R, SQL, Spark, C++]",AL,Database,Mid,no,2018-04-10,129784,yes


In [462]:
# what if we iterated differently?

tt=[]
for i in ex_2:
    tt.append(i)
pd.DataFrame(t)

'''
you would have ended up with what is 2 different dictionaries and they don't match
obviously. That is why you have NaN's. That is why the other loop was used. 
'''

,first_name,last_name,skills,state,specialty,experience,relocation,hire_date,Salary,healthcare
0,Margaret,Mcdonald,"[skLearn, Java, R, SQL, Spark, C++]",AL,Database,Mid,no,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[2018-04-10],[129784],[yes]


# We can parse the `Dates` if needed:

In [362]:
full_df=pd.concat([pd.DataFrame.from_dict(t[0],orient='index').T,
                   pd.DataFrame(t[1])],axis=1)

# Parse Dates:
full_df['hire_date']=pd.to_datetime(full_df['hire_date'],format='%Y-%d-%m')
full_df['hire_year']=full_df['hire_date'].dt.year

full_df['month']=full_df['hire_date'].dt.month_name()
full_df['day']=full_df['hire_date'].dt.day_name()
full_df

,first_name,last_name,skills,state,specialty,experience,relocation,hire_date,Salary,healthcare,hire_year,month,day
0,Margaret,Mcdonald,"[skLearn, Java, R, SQL, Spark, C++]",AL,Database,Mid,no,2018-10-04,129784,yes,2018,October,Thursday


# <font color=red>LIKE</font>, Share &
# <font color=red>SUB</font>scribe

# Citations & Help:

# ◔̯◔

https://stackoverflow.com/questions/19736080/creating-dataframe-from-a-dictionary-where-entries-have-different-lengths

https://aaronparecki.com/oauth-2-simplified/

https://medium.com/swlh/converting-nested-json-structures-to-pandas-dataframes-e8106c59976e

https://stackoverflow.com/questions/39899005/how-to-flatten-a-pandas-dataframe-with-some-columns-as-json/39906235

https://www.kaggle.com/jboysen/quick-tutorial-flatten-nested-json-in-pandas

https://stackoverflow.com/questions/43978022/parsing-date-time-strings-in-pandas-dataframe